In [74]:
# importando pacotes
from ucimlrepo import fetch_ucirepo 
import pandas
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, f1_score
import numpy

In [75]:
# CARREGANDO DADOS

# importando dataset
dataset = fetch_ucirepo(id=320) # 649 registros e 30 atributos
  
# coletando as informações
data_frame = dataset.data.original

In [76]:
# TRATANDO DADOS

print("PRÉ TRATAMENTO: ", len(data_frame))

# removendo colunas com muitos nulos
tolerancia = len(data_frame) * 0.7
data_frame = data_frame.dropna(axis=1, thresh=tolerancia)

# removendo duplicados
print("VALORES DUPLICADOS: ", data_frame.duplicated().sum())
data_frame = data_frame.drop_duplicates()

# convertendo colunas categóricas em valores inteiros
conversores = {}
for coluna in data_frame.columns:
    if (data_frame[coluna].dtype == type(object)):
        conversor = LabelEncoder()
        data_frame[coluna] = conversor.fit_transform(data_frame[coluna])
        conversores[coluna] = conversor

print("PÓS TRATAMENTO: ", len(data_frame))

PRÉ TRATAMENTO:  649
VALORES DUPLICADOS:  0
PÓS TRATAMENTO:  649


In [77]:
print(data_frame.columns)

Index(['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu',
       'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime',
       'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery',
       'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc',
       'Walc', 'health', 'absences', 'G1', 'G2', 'G3'],
      dtype='object')


In [78]:
# DIVIDINDO O DATASET TRATADO

atributos = data_frame.drop(["G1", "G2", "G3"], axis=1)
respostas = data_frame[["G1", "G2", "G3"]]

a_treino, a_teste, r_treino, r_teste = train_test_split(atributos, respostas, test_size=0.3, random_state=42)


In [79]:
print("TAMANHO DO DATASET TRATADO: ", len(data_frame))
print("CONJUNTO DE TREINO: ", a_treino.shape, " e ", r_treino.shape)
print("CONJUNTO DE TESTE: ", a_teste.shape, " e ", r_teste.shape)

TAMANHO DO DATASET TRATADO:  649
CONJUNTO DE TREINO:  (454, 30)  e  (454, 3)
CONJUNTO DE TESTE:  (195, 30)  e  (195, 3)


In [80]:
# CRIANDO PARÂMETROS PARA OS MODELOS

lista_parametros = [
    {"id": "1", "hidden_layer_sizes": (100, 50), "activation": "relu", "solver": "lbfgs", "max_iter": 700},
    {"id": "2", "hidden_layer_sizes": (100, 50), "activation": "relu", "solver": "lbfgs", "max_iter": 700},
    {"id": "3", "hidden_layer_sizes": (100, 50), "activation": "relu", "solver": "lbfgs", "max_iter": 700},
    {"id": "4", "hidden_layer_sizes": (100, 50), "activation": "relu", "solver": "lbfgs", "max_iter": 700},
]

In [81]:
# APLICANDO MODELO

# criando a lista de resultados finais
lista_resultados = []

for parametros in lista_parametros:
    # criando classificadores
    classificador_G1 = MLPClassifier(
        hidden_layer_sizes=parametros["hidden_layer_sizes"],
        activation=parametros["activation"],
        solver=parametros["solver"],
        max_iter=parametros["max_iter"],
        random_state=42
    )
    classificador_G2 = MLPClassifier(
        hidden_layer_sizes=parametros["hidden_layer_sizes"],
        activation=parametros["activation"],
        solver=parametros["solver"],
        max_iter=parametros["max_iter"],
        random_state=42
    )
    classificador_G3 = MLPClassifier(
        hidden_layer_sizes=parametros["hidden_layer_sizes"],
        activation=parametros["activation"],
        solver=parametros["solver"],
        max_iter=parametros["max_iter"],
        random_state=42
    )
    
    # treinando os modelos
    classificador_G1.fit(a_treino, r_treino.iloc[:, 0])
    classificador_G2.fit(a_treino, r_treino.iloc[:, 1])
    classificador_G3.fit(a_treino, r_treino.iloc[:, 2])

    # prevendo respostas
    r_previsao_G1 = classificador_G1.predict(a_teste)
    r_previsao_G2 = classificador_G2.predict(a_teste)
    r_previsao_G3 = classificador_G3.predict(a_teste)

    # calculando métricas
    acuracia_G1 = accuracy_score(r_teste.iloc[:, 0], r_previsao_G1)
    acuracia_G2 = accuracy_score(r_teste.iloc[:, 1], r_previsao_G2)
    acuracia_G3 = accuracy_score(r_teste.iloc[:, 2], r_previsao_G3)
    f1_G1 = f1_score(r_teste.iloc[:, 0], r_previsao_G1, average="weighted")
    f1_G2 = f1_score(r_teste.iloc[:, 1], r_previsao_G2, average="weighted")
    f1_G3 = f1_score(r_teste.iloc[:, 2], r_previsao_G3, average="weighted")

    # salvando resultados
    lista_resultados.append([parametros["id"], acuracia_G1, f1_G1, acuracia_G2, f1_G2, acuracia_G3, f1_G3])

In [82]:
# mostrando resultados
tabela_resultados = pandas.DataFrame(lista_resultados, columns=["Identificador", "AcuráciaG1", "F1-scoreG1", "AcuráciaG2", "F1-scoreG2", "AcuráciaG3", "F1-scoreG3"])
print(tabela_resultados)

  Identificador  AcuráciaG1  F1-scoreG1  AcuráciaG2  F1-scoreG2  AcuráciaG3  \
0             1    0.189744    0.180228    0.112821    0.112145    0.153846   
1             2    0.189744    0.180228    0.112821    0.112145    0.153846   
2             3    0.189744    0.180228    0.112821    0.112145    0.153846   
3             4    0.189744    0.180228    0.112821    0.112145    0.153846   

   F1-scoreG3  
0    0.157348  
1    0.157348  
2    0.157348  
3    0.157348  
